In [22]:
import numpy as np
import pandas as pd

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)
# Importing the dataset
dataset = pd.read_csv('data.csv') 
# print row names
print(dataset.columns)

Index(['Date', 'Recorder', 'Map', 'Player1', 'Player2', 'Player3', 'Player4',
       'vs', 'Player5', 'Player6', 'Player7', 'Player8', 'link'],
      dtype='object')


In [23]:
dataset["Map"] = dataset["Map"].str.lower()
dataset['Date'] = pd.to_datetime(dataset['Date'],format='%d.%m.%Y')

In [24]:
dataset=dataset.drop_duplicates(subset=['Date','Map'])
dataset.to_csv('new_data.csv',index=False)
dataset.drop(columns=['link','Recorder','vs'],   inplace=True)


In [25]:
map_weights = {
'ливень':5,
'кровавая жатва':5, 
'переход':3,
'мрачный карнавал':5,
'холодный ручей':4,
'похоронный звон':5,
'приход':5,
'смерть в воздухе':5,
'нет милосердию':5,
'вымерший центр':4,
'болотная лихорадка':4,
'последний рубеж':2,
'жертва':3,
'роковой полет':5}

def map_weight(map):
    return map_weights[map]

In [26]:
dataset["Map"].unique()

array(['ливень', 'кровавая жатва', 'переход', 'мрачный карнавал',
       'холодный ручей', 'похоронный звон', 'приход', 'смерть в воздухе',
       'нет милосердию', 'вымерший центр', 'болотная лихорадка',
       'последний рубеж', 'роковой полет', 'жертва'], dtype=object)

In [27]:
dataset["Map_weight"] = dataset["Map"].apply(map_weight)

In [28]:
# unique players from all 8 columns ["Player1","Player2","Player3","Player4","Player5","Player6","Player7","Player8"]
players = np.unique(dataset[["Player1","Player2","Player3","Player4","Player5","Player6","Player7","Player8"]].values)
players

array(['Бот', 'Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Марина',
       'Миша', 'Ондрей', 'Ондрей/Джун', 'Ондрей/Сеньор', 'Савва',
       'Сеньор', 'Юран'], dtype=object)

In [29]:
remove_players = ['Ондрей/Джун', 'Ондрей/Сеньор', 'Марина']
# drop rows with some chosen remove_players in any of the 8 columns
def drop_players(row):
    for player in remove_players:
        if player in row.values:
            return False
    return True

dataset.drop(dataset[~dataset.apply(drop_players, axis=1)].index, inplace=True)

players = np.unique(dataset[["Player1","Player2","Player3","Player4","Player5","Player6","Player7","Player8"]].values)

#  delete "Bot" from list
players = players[players != 'Бот']
players

array(['Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Миша', 'Ондрей',
       'Савва', 'Сеньор', 'Юран'], dtype=object)

In [30]:
# randonly assign 1 or 2 to TeamWon column
dataset["TeamWon"] = np.random.randint(1, 3, dataset.shape[0])
dataset

,Date,Map,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8,Map_weight,TeamWon
0,2024-03-09,ливень,Ден,Гриша,Ондрей,Юран,Сеньор,Савва,Данил,Бот,5,2
3,2024-03-03,кровавая жатва,Данил,Вадим,Сеньор,Савва,Гриша,Миша,Ден,Бот,5,2
6,2024-03-03,переход,Вадим,Сеньор,Юран,Гриша,Ден,Данил,Савва,Миша,3,1
8,2024-03-02,мрачный карнавал,Данил,Гриша,Сеньор,Савва,Вадим,Миша,Ваня,Бот,5,1
11,2024-02-25,холодный ручей,Ден,Вадим,Гриша,Бот,Юран,Сеньор,Ваня,Бот,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
205,2023-01-05,нет милосердию,Ден,Вадим,Миша,Сеньор,Джун,Данил,Юран,Ваня,5,1
210,2023-01-04,кровавая жатва,Ден,Данил,Ваня,Гриша,Миша,Ондрей,Джун,Вадим,5,2
211,2023-01-01,приход,Ваня,Сеньор,Миша,Ден,Данил,Ондрей,Джун,Гриша,5,1
212,2023-01-01,жертва,Сеньор,Джун,Ден,Вадим,Миша,Гриша,Ваня,Данил,3,2


In [31]:
#  add data with swapped players "Player1","Player2","Player3","Player4" with "Player5","Player6","Player7","Player8"
swapped = dataset.copy()
swapped[["Player1","Player2","Player3","Player4","Player5","Player6","Player7","Player8"]] = swapped[["Player5","Player6","Player7","Player8","Player1","Player2","Player3","Player4"]]

# combine original and swapped data
dataset = pd.concat([dataset, swapped])

# shuffle "Player1","Player2","Player3","Player4" and "Player5","Player6","Player7","Player8" columns


In [32]:
# Players from "Team 1" are Player1, Player2, Player3, Player4
# Players from "Team 2" are Player5, Player6, Player7, Player8
# TeamWon column is the team that won the game
#  assign to player in which team he was

for player in players:
    dataset[player] = 0
    dataset.loc[(dataset['Player1'] == player) | (dataset['Player2'] == player) | (dataset['Player3'] == player) | (dataset['Player4'] == player), player] = 1
    dataset.loc[(dataset['Player5'] == player) | (dataset['Player6'] == player) | (dataset['Player7'] == player) | (dataset['Player8'] == player), player] = 2

# if player Player4=="Bot" and Player8=="Bot" then "Bot" column should be equal to 0
# dataset.loc[(dataset['Player4'] == "Bot") & (dataset['Player8'] == "Bot"), 'Bot'] = 0

In [33]:
dataset

,Date,Map,Player1,Player2,Player3,Player4,Player5,Player6,Player7,Player8,Map_weight,TeamWon,Вадим,Ваня,Гриша,Данил,Ден,Джун,Миша,Ондрей,Савва,Сеньор,Юран
0,2024-03-09,ливень,Ден,Гриша,Ондрей,Юран,Сеньор,Савва,Данил,Бот,5,2,0,0,1,2,1,0,0,1,2,2,1
3,2024-03-03,кровавая жатва,Данил,Вадим,Сеньор,Савва,Гриша,Миша,Ден,Бот,5,2,1,0,2,1,2,0,2,0,1,1,0
6,2024-03-03,переход,Вадим,Сеньор,Юран,Гриша,Ден,Данил,Савва,Миша,3,1,1,0,1,2,2,0,2,0,2,1,1
8,2024-03-02,мрачный карнавал,Данил,Гриша,Сеньор,Савва,Вадим,Миша,Ваня,Бот,5,1,2,2,1,1,0,0,2,0,1,1,0
11,2024-02-25,холодный ручей,Ден,Вадим,Гриша,Бот,Юран,Сеньор,Ваня,Бот,4,1,1,2,1,0,1,0,0,0,0,2,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,2023-01-05,нет милосердию,Джун,Данил,Юран,Ваня,Ден,Вадим,Миша,Сеньор,5,1,2,1,0,1,2,1,2,0,0,2,1
210,2023-01-04,кровавая жатва,Миша,Ондрей,Джун,Вадим,Ден,Данил,Ваня,Гриша,5,2,1,2,2,2,2,1,1,1,0,0,0
211,2023-01-01,приход,Данил,Ондрей,Джун,Гриша,Ваня,Сеньор,Миша,Ден,5,1,0,2,1,1,2,1,2,1,0,2,0
212,2023-01-01,жертва,Миша,Гриша,Ваня,Данил,Сеньор,Джун,Ден,Вадим,3,2,2,1,1,1,2,2,1,0,0,2,0


In [34]:
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# split the dataset into train and test
train, test = train_test_split(dataset, test_size=0.1, random_state=42)

# select the columns to predict
y_train = train['TeamWon']-1 
y_test = test['TeamWon']-1

# select the columns to train the model
X_train = train[players]
X_test = test[players]

#  scale the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)



In [35]:
y_test

63     1
133    1
172    0
193    0
118    0
103    0
92     1
154    1
21     1
185    1
64     0
161    1
211    0
45     1
31     1
142    0
118    0
37     0
125    0
190    0
148    1
Name: TeamWon, dtype: int64

In [36]:
#  create a model
model = keras.Sequential([
    keras.layers.Dense(100, input_shape=(len(players),), activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
# compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
# train the model
model.fit(X_train, y_train, epochs=10)

Epoch 1/10
6/6 [==============================] - 1s 7ms/step - loss: 0.7003 - accuracy: 0.5079
Epoch 2/10
6/6 [==============================] - 0s 6ms/step - loss: 0.6698 - accuracy: 0.6085
Epoch 3/10
6/6 [==============================] - 0s 8ms/step - loss: 0.6515 - accuracy: 0.6508
Epoch 4/10
6/6 [==============================] - 0s 6ms/step - loss: 0.6366 - accuracy: 0.6984
Epoch 5/10
6/6 [==============================] - 0s 7ms/step - loss: 0.6233 - accuracy: 0.7090
Epoch 6/10
6/6 [==============================] - 0s 5ms/step - loss: 0.6136 - accuracy: 0.7037
Epoch 7/10
6/6 [==============================] - 0s 6ms/step - loss: 0.6076 - accuracy: 0.6878
Epoch 8/10
6/6 [==============================] - 0s 6ms/step - loss: 0.5942 - accuracy: 0.7143
Epoch 9/10
6/6 [==============================] - 0s 5ms/step - loss: 0.5873 - accuracy: 0.7143
Epoch 10/10
6/6 [==============================] - 0s 7ms/step - loss: 0.5787 - accuracy: 0.7196


In [37]:
# test the model
test_loss, test_acc = model.evaluate(X_test,  y_test, verbose=2)

1/1 - 0s - loss: 0.8098 - accuracy: 0.4762 - 83ms/epoch - 83ms/step


In [38]:
# now we can predict the result of a game based on the present players
# let's say we have the following players
# 'Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Миша', 'Ондрей', 'Савва', 'Сеньор', 'Юран'
present_players = ['Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун','Сеньор','Юран']
# we need to test all possible teams with 4 players with no repetitions and save the result to list

In [39]:
all_results = []
# add bots to the present players if there are less than 8 players
while len(present_players) < 8:
      present_players = present_players + ["Бот"]
print ('present_players are', present_players)
for i in range(0, len(present_players)):
    for j in range(i+1, len(present_players)):
        for k in range(j+1, len(present_players)):
            for l in range(k+1, len(present_players)):
                                team1 = [present_players[i], present_players[j], present_players[k], present_players[l]]
                                team2 = list(set(present_players) - set(team1))
                                if len(team2) < 4: 
                                    team2 = team2 + ["Бот"] * (4 - len(team2))
                                # create a new row with the present players
                                row = pd.DataFrame([team1 + team2], columns=["Player1","Player2","Player3","Player4","Player5","Player6","Player7","Player8"])
                                # assign 1 or 2 to each player based on the team
                                for player in players:
                                    row[player] = 0
                                    if player in team1:
                                        row[player] = 1
                                    if player in team2:
                                        row[player] = 2
                                    
                                # scale the data
                                row = scaler.transform(row[players])
                                # predict the result
                                result = model.predict(row, verbose=0)
                                all_results.append((team1, team2, result))
# sort the results
all_results.sort(key=lambda x: x[2], reverse=True)

present_players are ['Вадим', 'Ваня', 'Гриша', 'Данил', 'Ден', 'Джун', 'Сеньор', 'Юран']


In [40]:
#  take the middle of the list
middle = len(all_results) // 2
print(all_results[middle])

(['Вадим', 'Ваня', 'Джун', 'Юран'], ['Ден', 'Гриша', 'Сеньор', 'Данил'], array([[0.35434645]], dtype=float32))


In [41]:
# take closest to 50%  result
closest = min(all_results, key=lambda x: abs(x[2] - 0.5))
print(closest)


(['Данил', 'Ден', 'Сеньор', 'Юран'], ['Ваня', 'Гриша', 'Вадим', 'Джун'], array([[0.49314967]], dtype=float32))
